In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Veri yükleme
df = pd.read_csv("BankNote_Authentication.csv")
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

X, y = df.iloc[:, :-1], df.iloc[:, -1]
print("X type is " + str(type(X)))
print("X shape is " + str(X.shape))
print("y type is " + str(type(y)))
print("y shape is " + str(y.shape))

X = X.to_numpy()
y = y.to_numpy().reshape(-1, 1)

print("X type is " + str(type(X)))
print("X shape is " + str(X.shape))
print("y type is " + str(type(y)))
print("y shape is " + str(y.shape))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("X train shape is " + str(X_train.shape))
print("X test shape is " + str(X_test.shape))
print("y train shape is " + str(y_train.shape))
print("y test shape is " + str(y_test.shape))

def initialize_parameters(n_x, n_h1, n_h2, n_y=1):
    np.random.seed(42)
    W1 = np.random.randn(n_h1, n_x) * 0.01
    b1 = np.zeros((n_h1, 1))
    W2 = np.random.randn(n_h2, n_h1) * 0.01
    b2 = np.zeros((n_h2, 1))
    W3 = np.random.randn(n_y, n_h2) * 0.01
    b3 = np.zeros((n_y, 1))
    
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2, "W3": W3, "b3": b3}
    return parameters

def relu(Z):
    return np.maximum(0, Z)

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def forward_propagation(X, parameters):
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    # İlk gizli katman
    Z1 = np.dot(W1, X.T) + b1
    A1 = relu(Z1)
    # İkinci gizli katman
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    # Çıkış katmanı
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    cache = {
        "Z1": Z1,
        "A1": A1,
        "Z2": Z2,
        "A2": A2,
        "Z3": Z3,
        "A3": A3
    }

    return A3, cache

def compute_cost(A3, Y):
    m = Y.shape[0]
    Y = Y.T  # Boyut uyumsuzluğunu gidermek için Y'yi transpoze ediyoruz
    
    cost = -np.sum(Y * np.log(A3) + (1 - Y) * np.log(1 - A3)) / m
    cost = float(np.squeeze(cost))
    return cost

def backpropagation(X, Y, cache, parameters):
    m = X.shape[0]  # Örnek sayısı
    Y = Y.T  # Boyutları uyumlu hale getirmek için
    
    A1, A2, A3 = cache['A1'], cache['A2'], cache['A3']
    
    # Çıkış katmanı (A3)
    dZ3 = A3 - Y
    dW3 = np.dot(dZ3, A2.T) / m
    db3 = np.sum(dZ3, axis=1, keepdims=True) / m
    
    # İkinci gizli katman (A2)
    dA2 = np.dot(parameters['W3'].T, dZ3)
    dZ2 = dA2 * (A2 > 0)  # ReLU türevi
    dW2 = np.dot(dZ2, A1.T) / m
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m
    
    # İlk gizli katman (A1)
    dA1 = np.dot(parameters['W2'].T, dZ2)
    dZ1 = dA1 * (A1 > 0)  # ReLU türevi
    dW1 = np.dot(dZ1, X) / m
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m

    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2, "dW3": dW3, "db3": db3}
    return grads

def update_parameters(parameters, grads, learning_rate=0.01):  
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    dW3 = grads["dW3"]
    db3 = grads["db3"]

    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W3 -= learning_rate * dW3
    b3 -= learning_rate * db3

    parameters = {
        "W1": W1,
        "b1": b1,
        "W2": W2,
        "b2": b2,
        "W3": W3,
        "b3": b3
    }

    return parameters

def nn_model(X, Y, n_x, n_h1, n_h2, n_y, n_steps=1000, print_cost=True):
    parameters = initialize_parameters(n_x, n_h1, n_h2, n_y)

    for i in range(0, n_steps):
        A3, cache = forward_propagation(X, parameters)
        cost = compute_cost(A3, Y)
        grads = backpropagation(X, Y, cache, parameters)
        parameters = update_parameters(parameters, grads)

        if print_cost and i % 100 == 0:
            print("Cost after iteration %i: %f" % (i, cost))

    return parameters

def predict(parameters, X):
    A3, cache = forward_propagation(X, parameters)
    predictions = (A3 > 0.5).astype(int)
    return predictions

# Modeli eğit
parameters = nn_model(X_train, y_train, X_train.shape[1], n_h1=6, n_h2=6, n_y=1, n_steps=1000)

# Tahmin yap ve performansı değerlendir
predictions = predict(parameters, X_test)

y_true = y_test.flatten()  
y_pred = predictions.flatten() 
acc = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='binary') 
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')
conf_matrix = confusion_matrix(y_true, y_pred)
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

# Farklı parametrelerle grid arama
parameters_n_h = [i for i in range(3, 11)]
parameters_n_steps = [i for i in range(100, 1100, 100)]
results = []

for n_h1 in parameters_n_h:
    for n_step in parameters_n_steps:
        parameters = nn_model(X_train, y_train, X_train.shape[1], n_h1=n_h1, n_h2=n_h1, n_y=1, n_steps=n_step, print_cost=False)
        predictions = predict(parameters, X_test)
        acc = accuracy_score(y_test.flatten(), predictions.flatten())
        results.append({"n_h1": n_h1, "n_h2": n_h1, "n_step": n_step, "accuracy": acc})
        print(f"n_h1: {n_h1}, n_h2: {n_h1}, n_step: {n_step}, acc: {acc}")

# En iyi sonucu bulma
best_result = max(results, key=lambda x: x["accuracy"])
print(f"\nEn iyi sonuç: {best_result}")


X type is <class 'pandas.core.frame.DataFrame'>
X shape is (1372, 4)
y type is <class 'pandas.core.series.Series'>
y shape is (1372,)
X type is <class 'numpy.ndarray'>
X shape is (1372, 4)
y type is <class 'numpy.ndarray'>
y shape is (1372, 1)
X train shape is (1097, 4)
X test shape is (275, 4)
y train shape is (1097, 1)
y test shape is (275, 1)
Cost after iteration 0: 0.693148
Cost after iteration 100: 0.690750
Cost after iteration 200: 0.689297
Cost after iteration 300: 0.688415
Cost after iteration 400: 0.687880
Cost after iteration 500: 0.687554
Cost after iteration 600: 0.687357
Cost after iteration 700: 0.687236
Cost after iteration 800: 0.687163
Cost after iteration 900: 0.687118


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.p

Accuracy: 0.5564
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Confusion Matrix:
[[153   0]
 [122   0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.56      1.00      0.71       153
           1       0.00      0.00      0.00       122

    accuracy                           0.56       275
   macro avg       0.28      0.50      0.36       275
weighted avg       0.31      0.56      0.40       275

n_h1: 3, n_h2: 3, n_step: 100, acc: 0.5563636363636364
n_h1: 3, n_h2: 3, n_step: 200, acc: 0.5563636363636364
n_h1: 3, n_h2: 3, n_step: 300, acc: 0.5563636363636364
n_h1: 3, n_h2: 3, n_step: 400, acc: 0.5563636363636364
n_h1: 3, n_h2: 3, n_step: 500, acc: 0.5563636363636364
n_h1: 3, n_h2: 3, n_step: 600, acc: 0.5563636363636364
n_h1: 3, n_h2: 3, n_step: 700, acc: 0.5563636363636364
n_h1: 3, n_h2: 3, n_step: 800, acc: 0.5563636363636364
n_h1: 3, n_h2: 3, n_step: 900, acc: 0.5563636363636364
n_h1: 3, n_h2: 3, n_step: 1000, acc: 0.556363